In [19]:
from urllib.parse import quote, unquote, quote_plus, urlencode

from base64 import b64encode
import json
import time

import requests

from scrapy.http import HtmlResponse

In [50]:
def get_new_token(response):
    token = response.headers['Set-Cookie'].split(";")[0].split('=')[1]
#     print("got token", token)
#     print()
    return token

In [3]:
def gen_lot_xml(order_id, buy_lot_id, buy_lot_point_id, price):
    return (
        f'<?xml version="1.0" encoding="UTF-8"?>'
        f'<priceoffer>'
        f'<trd_buy_id>{order_id}</trd_buy_id>'
        f'<trd_buy_lot_id>{buy_lot_id}</trd_buy_lot_id>'
        f'<trd_buy_lot_point_id>{buy_lot_point_id}</trd_buy_lot_point_id>'
        f'<price>{price}</price>'
        f'</priceoffer>'
    )

In [4]:
def gen_xml_data(prices):
    res = {}
    
    for price in prices:
        res[f'xmlData[{price["buy_lot_point_id"]}]'] = price['xml_string']
        
    return res

In [5]:
def sign(string):
    cert_path='/home/necuk/Downloads/key2/GOSTKNCA_de113b5aa34a4d38ee8836c3e2ae2e3c8e23d06c.p12'
    with open (cert_path, 'rb') as f:
        cert = b64encode(f.read()).decode('utf-8')
    
    data = {
        "version": "2.0",
        "method": "cms.sign",
        "params": {
            "data": b64encode(string.encode('utf-8')).decode('utf-8'),
            "p12array": [
                {
                    "alias": "",
                    "p12": cert,
                    "password":"baRsa01022020"
                }
            ]
        }
    }
    
    
    headers = {
        'Content-Type': 'application/json'
    }
    
    r = requests.post(
        url='http://127.0.0.1:14579', 
        data=json.dumps(data),
    #     data = data,
        headers=headers
    )

    print(r.status_code)
    
    return json.loads(r.text)['cms']

In [6]:
def gen_prices_request_data(prices):
    formdata = {
        f"offer[{record['app_lot_id']}][{record['buy_lot_point_id']}][price]": record['price']
        for record in prices
    }
    formdata_encoded = 'formData=' + urlencode(formdata)
    
    xml = gen_xml_data(prices)
    
    signatures = {}
    for p in prices:
        cms = sign(p['xml_string'])
        signatures[f'signData[{p["buy_lot_point_id"]}]'] = cms
        
    return urlencode(xml, safe='+') + '&' + urlencode(signatures) + '&' + formdata_encoded

In [45]:
def prices_1(token):

    cookies = {
        'show_filter_app': 'block',
    #     'ci_session': self.get_token_scrapy(response)
    #     'ci_session': get_new_token(response)
        'ci_session': token
    }

    headers = {
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': self.user_agent,
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Dest': 'document',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Linux"',
        'Referer': f'https://v3bl.goszakup.gov.kz/ru/application/docs/{self.order_id}/{self.application_id}',
        'Accept-Language': 'en-US,en;q=0.9',
    }

    # request = scrapy.Request(
    #     method='GET',
    #     url=f'https://v3bl.goszakup.gov.kz/ru/application/priceoffers/{self.order_id}/{self.application_id}', 
    #     headers=headers, 
    #     cookies=cookies,
    #     dont_filter=True,
    #     callback=self.prices_3
    # )

    r = requests.get(
        url=f'https://v3bl.goszakup.gov.kz/ru/application/priceoffers/{self.order_id}/{self.application_id}', 
        headers=headers, 
        cookies=cookies,
        verify=False
    )

    print('r1', r.status_code)
    
    return r

In [31]:
def prices_1(token):

    cookies = {
        'show_filter_app': 'block',
    #     'ci_session': self.get_token_scrapy(response)
    #     'ci_session': get_new_token(response)
        'ci_session': token
    }

    headers = {
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': self.user_agent,
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Dest': 'document',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Linux"',
        'Referer': f'https://v3bl.goszakup.gov.kz/ru/application/docs/{self.order_id}/{self.application_id}',
        'Accept-Language': 'en-US,en;q=0.9',
    }

    r = requests.get(
        url=f'https://v3bl.goszakup.gov.kz/ru/application/priceoffers/{self.order_id}/{self.application_id}', 
        headers=headers, 
        cookies=cookies,
        verify=False
    )

    print('r1', r.status_code)
    
    return r

In [39]:
def gen_prices(r):
    response = HtmlResponse(url="", body=r.content)
    rows = response.xpath('.//div[@class="panel panel-default"]/table//tr')[1::2]

    prices = []

    for row in rows:
        start_price = row.xpath('./td[6]/text()').get()
        new_price = round(float(start_price.replace(' ', '')) * (1 - self.price_discount), 2)

        record = {
            'price': new_price,
            'buy_lot_id': row.xpath('./td[9]/div/input/@trd_lot_id').get(),
            'buy_lot_point_id': row.xpath('./td[9]/div/input/@buy_lot_point_id').get(),
            'app_lot_id': row.xpath('./td[9]/div/input/@app_lot_id').get()
        }

        record['xml_string'] = gen_lot_xml(self.order_id, record['buy_lot_id'], record['buy_lot_point_id'], record['price'])

        prices.append(record)

#     len(rows), len(self.prices)
    print(f"got {len(prices)} prices")
    
    return prices

In [40]:
def prices_2(token, prices):

    cookies = {
        'show_filter_app': 'block',
    #         'ci_session': self.get_token_scrapy(response)
    #     'ci_session': get_new_token(r)
        'ci_session': token
    }

    headers = {
        'Connection': 'keep-alive',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'X-Requested-With': 'XMLHttpRequest',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
        'sec-ch-ua-platform': '"Linux"',
        'Origin': 'https://v3bl.goszakup.gov.kz',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Referer': f'https://v3bl.goszakup.gov.kz/ru/application/priceoffers/{self.order_id}/{self.application_id}',
        'Accept-Language': 'en-US,en;q=0.9',
    }

    data = {
        f"offer[{record['app_lot_id']}][{record['buy_lot_point_id']}][price]": record['price']
#         for record in self.prices
        for record in prices
    }

    r2 = requests.post(
        url=f'https://v3bl.goszakup.gov.kz/ru/application/ajax_check_priceoffers/{self.order_id}/{self.application_id}', 
        headers=headers, 
        cookies=cookies, 
        data=data,
        verify=False
    )

    print('r2', r2.status_code, r2.text)
    
    return r2

In [33]:
def prices_submit(token, prices):

    cookies = {
        'show_filter_app': 'block',
        'ci_session': token
    #     'ci_session': get_new_token(r2)
    }

    headers = {
        'Connection': 'keep-alive',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'X-Requested-With': 'XMLHttpRequest',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
        'sec-ch-ua-platform': '"Linux"',
        'Origin': 'https://v3bl.goszakup.gov.kz',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
    #     'Referer': 'https://v3bl.goszakup.gov.kz/ru/application/priceoffers/7196072/37068725',
        'Accept-Language': 'en-US,en;q=0.9',
    }

    data = gen_prices_request_data(prices)

    r3 = requests.post(
        f'https://v3bl.goszakup.gov.kz/ru/application/ajax_add_priceoffers/{self.order_id}/{self.application_id}', 
        headers=headers, cookies=cookies, data=data, verify=False
    )

    if len(r3.text) < 200:
        print('submit', r3.status_code, r3.text)
    else:
        print('submit',r3.status_code, "oops unlogin")
        
    return r3

In [59]:
class self:
    price_discount = 0.1
    order_id = '7196072'
    application_id = '37086538'
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'

In [54]:
token = "XjVcZlJlVjkBLlcmAWxTYgMwVmEDCgUgWCMFN1VzU3JXPAM8UG4BUgBjCW1bBAF2DmFTIgNuW20BNldgBg8CJ1hjBDdcawAxVTMEbFRmVzleZ1xoUmBWMgEzV2ABZVNrAzBWPgM1BWFYZwVjVWNTYldtA2JQYgFsAD4JbFtrATcOaVNnA2ZbawE0VzgGagJkWDUEYVw6AGdVYARnVDBXbl43XGlSNVY0AWVXMwFmUzEDMlY4AzIFZlgyBTdVM1NjVwkDcVA7AX4AMAk4W2gBbg4GU3MDPVsrAV1XOwY3AmFYIgQ2XHoAIVULBHZUP1coXm5ca1JrVl8Bd1dlAXhTYwMuVmsDeAUyWAwFcFU7U3JXbwNiUDABNwBWCStbLQEnDj9TIwMLWzoBZVc%2FBj0CcVgMBHFcMgAhVW0EZVQ0V29eblwAUnNWTgE6Vy8BP1M%2FA2xWOgN5BTdYfgViVSBTKVcNA2JQMQE2ACoJRVsxAToOL1MpA3RbIwE6V2wGDAIzWGQEelwpABNVJwQkVGhXPl4DXDlSM1ZIATxXIQF5U2YDM1ZsA3gFMVhmBXJVKFNKVx0DB1BNAUEAJgkpWzQBPQ4xUzQDdFscAWdXOQY4AmpYeQRzXEoAOlUlBDtUaVc%2BXntcZVJoVi0BZVd7AWJTawM0Vm8DeAU3WGEFclVTU2BXMwMyUHIBZAAlCTxbawFjDnRTYgNiWwcBIFdhBiACP1hhBGBcMwAOVXUEOFRlVyheIFwDUjBWYAEhVzwBIFM6A3RWIgMKBSBYawU7VTpTMFdjA2dQOAE5ADgJOVtsAWEOalNqAyk%3D"

In [51]:
# r1 = prices_1(token)
# time.sleep(1)

# prices = gen_prices(r1)
# time.sleep(1)

# r2 = prices_2(get_new_token(r1), prices)
# time.sleep(0.5)

# r3 = prices_submit(get_new_token(r2), prices)

In [57]:
token="XTYPNQYxBmkHKA9%2BBmtWZ1ZlVWIFDAUgCHMCMFN1UnMCaQc4AT9UBwBjCm4GWQZxVToDcgZrAzVRZgw7UFkKLw0yVTNdaVQzBWMFZQY0Vj5dMg9tBjUGZwdkDzgGMlZnVmVVPgVlBWQIMAJgUzFSYQJhBzEBaFQ7ADMKaAZgBjZVOQMwBmUDMlFrDDVQNQo1DTFVZ11uVDAFYgU0BmNWYl1gD2kGYQZrBzAPOwY0VmRWM1U%2BBWMFZwg0AmRTZVJhAlwHdQFqVCsAMAo7BjUGaVVdAyMGOANzUQ0MYFBhCmkNd1VnXXtUdQVbBXcGbVYpXW0POAY%2FBg8HcQ89Bn9WZlZ7VWgFfgUyCFwCd1M9UnMCOgdmAWFUYgBWCigGcAYgVWQDcwYOA2JRNQxkUGsKeQ1ZVSBdM1R1BT0FZAZmVm5dbQ9TBicGHgc8D3cGOFY6VjlVOQV%2FBTcILgJlUyZSKAJYB2YBYFRjACoKRgZsBj1VdAN5BnEDe1FqDDdQWgo7DTFVK10oVEcFdwUlBjpWP10AD2oGZwYYBzoPeQZ%2BVmNWZlVvBX4FMQg2AnVTLlJLAkgHAwEcVBQAJgoqBmkGOlVqA2QGcQNEUTcMYlBuCmINLFUiXUtUbgV1BToGO1Y%2FXXgPNgY8Bn0HYw8jBmVWblZhVWwFfgU3CDECdVNVUmECZgc2ASNUMQAlCj8GNgZkVS8DMgZnA19RcAw6UHYKNw00VTFdMlRaBSUFOQY3VildIw9QBmQGMAcnD2QGJ1Y%2FViFVIQUMBSAIOwI8UzxSMQI2B2MBaVRsADgKPgY9BmZVNAM6Biw%3D"

In [60]:
r1 = prices_1(token)
time.sleep(1)

prices = gen_prices(r1)
time.sleep(1)

token = get_new_token(r1)

for price in prices:
    r2 = prices_2(token, [price])
    time.sleep(0.5)

    token = get_new_token(r2)
    r3 = prices_submit(token, [price])
    time.sleep(0.5)
    
    token = get_new_token(r3)

/home/necuk/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'v3bl.goszakup.gov.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


r1 200
got 3 prices


/home/necuk/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'v3bl.goszakup.gov.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


r2 200 {"status":1}
200


/home/necuk/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'v3bl.goszakup.gov.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

submit 200 {"status":1}


/home/necuk/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'v3bl.goszakup.gov.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


r2 200 {"status":1}
200


/home/necuk/.local/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'v3bl.goszakup.gov.kz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


submit 200 {"status":1}


In [61]:
import random

In [75]:
tt = [1, 2, 3, 4, 5]

In [76]:
tt = [str(i) for i in tt]

In [77]:
random.choice(tt)

'2'

In [78]:
tt.remove('1')

In [80]:
tt

['2', '3', '4', '5']